# Create layout based on cluster information
* Load network in GraphML
* Convert string group information to integers
* Apply

In [1]:
from graph_tool.all import *

#ORIGINAL = "nest2.graphml"

# ORIGINAL = "nest2_large.graphml"
FILTERED = "nest-filtered.graphml" # Score > 0.6

ORIGINAL = "sig_with_groups.graphml"

FILTERED2 = "temp-prepared.graphml"

In [2]:
nest = load_graph(FILTERED2)
nest

<Graph object, undirected, with 1085 vertices and 14561 edges, 61 internal vertex properties, 12 internal edge properties, at 0x7fe3517afdf0>

In [3]:
members = nest.vertex_properties["layoutMember"]

value_set = set()
for member in members:
    value_set.add(member)

v2idx = {}
idx = 1
for entry in value_set:
    v2idx[entry] = idx
    idx += 1

# Group names to integers
print(v2idx)

{'': 1, 'Group:NEST:95': 2, 'Group:NEST:294': 3, 'Group:NEST:235': 4, 'Group:NEST:204': 5, 'Group:NEST:299': 6, 'Group:NEST:256': 7, 'Group:NEST:224': 8, 'Group:NEST:244': 9, 'Group:NEST:191': 10, 'Group:NEST:291': 11, 'Group:NEST:250': 12, 'Group:NEST:202': 13, 'Group:NEST:272': 14, 'Group:NEST:126': 15, 'Group:NEST:240': 16, 'Group:NEST:210': 17, 'Group:NEST:310': 18, 'Group:NEST:100': 19, 'Group:NEST:150': 20, 'Group:NEST:298': 21, 'Group:NEST:181': 22, 'Group:NEST:179': 23, 'Group:NEST:128': 24, 'Group:NEST:111': 25, 'Group:NEST:174': 26, 'Group:NEST:337': 27, 'Group:NEST:211': 28, 'Group:NEST:90': 29, 'Group:NEST:37': 30, 'Group:NEST:56': 31, 'Group:NEST:173': 32, 'Group:NEST:153': 33, 'Group:NEST:205': 34, 'Group:NEST:109': 35, 'Group:NEST:309': 36, 'Group:NEST:221': 37, 'Group:NEST:149': 38, 'Group:NEST:103': 39, 'Group:NEST:343': 40, 'Group:NEST:125': 41, 'Group:NEST:249': 42, 'Group:NEST:300': 43, 'Group:NEST:146': 44, 'Group:NEST:283': 45, 'Group:NEST:287': 46, 'Group:NEST:22

In [7]:
# New attribute to show membership as an integer
groups = nest.new_vertex_property("int32_t")
nodes = nest.vertices()
for node in nodes:
    groups[node] = v2idx[members[node]]
    
# Name it "layoutIndex"
nest.vp.layoutIndex = groups

# Edge scores to be used for layout
score = nest.edge_properties["Score"]
nest.vp.layoutIndex

<VertexPropertyMap object with value type 'int32_t', for Graph 0x7fe3517afdf0, at 0x7fe3517c1130>

In [5]:
# pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=1000, eweight=score, C=0.5, multilevel=True)

# Dense cluster
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=0.01, mu=10000, eweight=score, C=0.5, multilevel=True)

# Clusters are closer to each other
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5, multilevel=True)

# Final for root
#pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5)

# For sig
pos1 = sfdp_layout(nest, groups=groups, vweight=groups, gamma=10000, mu=10000, eweight=score, C=0.5)


# graph_draw(nest, pos=pos1, vertex_fill_color=groups, edge_color=[0.7, 0.7,0.7, 0.3], output="graph-draw-sfdp-all.pdf")

RuntimeError: No static implementation was found for the desired routine. This is a graph_tool bug. :-( Please submit a bug report at http://graph-tool.skewed.de/issues. What follows is debug information.

Action: std::_Bind<get_community_network_edges_dispatch (std::_Placeholder<1>, std::reference_wrapper<boost::adj_list<unsigned long> >, boost::adj_edge_index_property_map<unsigned long>, std::_Placeholder<2>, boost::any, std::_Placeholder<3>, boost::any)>

Arg 1: std::reference_wrapper<boost::undirected_adaptor<boost::adj_list<unsigned long> > >

Arg 2: boost::checked_vector_property_map<int, boost::typed_identity_property_map<unsigned long> >

Arg 3: boost::checked_vector_property_map<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, boost::adj_edge_index_property_map<unsigned long> >



In [ ]:
# Adjust position (For Cytoscape Desktop)
x = nest.new_vertex_property("double")
y = nest.new_vertex_property("double")

# Original positions are too dense for CYtoscape.
SCALING = 100

vs = nest.vertices()
for v in vs:
    p = pos1[v]
    x[v] = p[0] * SCALING
    y[v] = p[1] * SCALING

nest.vp.xpos = x
nest.vp.ypos = y

In [ ]:
# list(nest.vertex_properties.keys())
nest.save("export-scaled-full-sig3.graphml", fmt="graphml")